import pandas as pd

# Чтение локального файла
df = pd.read_csv("processed_asthma.csv")

# Просмотр первых 5 строк
df.head()


# Проверка структуры данных
df.info()

# Основная статистика для числовых столбцов
df.describe()


# Подсчет пропущенных значений
missing = df.isnull().sum()

# Процент пропусков
missing_percent = (missing / len(df)) * 100

# Создаем таблицу с результатами
missing_df = pd.DataFrame({
    'Пропущенные значения': missing,
    'Процент %': missing_percent
})

missing_df


# Выбираем числовые столбцы
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Расчет IQR для выявления выбросов
Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Подсчет выбросов для каждого числового столбца
outliers = ((df[numeric_cols] < (Q1 - 1.5 * IQR)) | (df[numeric_cols] > (Q3 + 1.5 * IQR))).sum()
outliers_percent = (outliers / len(df)) * 100

outliers_df = pd.DataFrame({
    'Выбросы': outliers,
    'Процент %': outliers_percent
})
outliers_df


# Создаем сводную таблицу с пропусками и выбросами
metrics_df = pd.DataFrame({
    'Пропущенные значения %': (df.isnull().sum() / len(df)) * 100,
    'Выбросы %': ((df[numeric_cols] < (Q1 - 1.5*IQR)) | (df[numeric_cols] > (Q3 + 1.5*IQR))).sum() / len(df) * 100
})

# Для категориальных столбцов добавляем только пропуски
for col in df.select_dtypes(include=['object']).columns:
    if col not in metrics_df.index:
        metrics_df.loc[col] = [df[col].isnull().sum() / len(df) * 100, 0]

metrics_df = metrics_df.sort_index()
metrics_df

